In [85]:
import pandas as pd
import numpy as np
from pathlib import Path
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [86]:
try:
    project_root = Path.cwd().parent
    clean_data_path = project_root / "data" / "processed" / "credit_card_clients_clean.csv"
    df = pd.read_csv(clean_data_path)
    print(f"DataFrame limpio cargado desde: {clean_data_path}")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo: {clean_data_path}")
    df = None


DataFrame limpio cargado desde: /Users/edusant/Desktop/personal/blue_tab/proyecto-riesgo-crediticio/data/processed/credit_card_clients_clean.csv


In [88]:
if df is not None:
    df_features = df.copy()

    # Definir grupos de columnas para facilitar los cálculos
    bill_amt_cols = ['bill_amt_sept', 'bill_amt_aug', 'bill_amt_july', 'bill_amt_june', 'bill_amt_may', 'bill_amt_april']
    pay_amt_cols = ['pay_amt_sept', 'pay_amt_aug', 'pay_amt_july', 'pay_amt_june', 'pay_amt_may', 'pay_amt_april']
    pay_status_cols = ['pay_sept', 'pay_aug', 'pay_july', 'pay_june', 'pay_may', 'pay_april']

    # 3.1. Ratios de Salud Financiera
    # Se calcula la utilización del crédito y la proporción de pago para cada mes.
    epsilon = 1e-6  # Para evitar división por cero
    for month in ['sept', 'aug', 'july', 'june', 'may', 'april']:
        df_features[f'utilization_{month}'] = df_features[f'bill_amt_{month}'] / (df_features['limit_bal'] + epsilon)
        df_features[f'payment_ratio_{month}'] = df_features[f'pay_amt_{month}'] / (df_features[f'bill_amt_{month}'] + epsilon)

    # 3.2. Agregados de Historial Financiero
    # Se crean métricas agregadas para resumir el comportamiento a lo largo de los 6 meses.
    df_features['bill_amt_avg'] = df_features[bill_amt_cols].mean(axis=1)
    df_features['bill_amt_std'] = df_features[bill_amt_cols].std(axis=1)
    df_features['pay_amt_avg'] = df_features[pay_amt_cols].mean(axis=1)

    # 3.3. Características de Tendencia
    # Se calcula la pendiente para observar tendencias en facturación y pagos.
    def calculate_slope(row, cols):
        # CORRECCIÓN: Se convierten los valores a tipo float para evitar errores de tipo.
        y = row[cols].values.astype(float)
        x = np.array(range(len(y)))
        # Se ajusta un polinomio de grado 1 (una línea) y se devuelve la pendiente.
        slope, _ = np.polyfit(x, y, 1)
        return slope

    # Se aplica la función a las columnas de interés (en orden cronológico inverso).
    df_features['bill_amt_slope'] = df_features.apply(lambda row: calculate_slope(row, bill_amt_cols[::-1]), axis=1)
    df_features['pay_status_slope'] = df_features.apply(lambda row: calculate_slope(row, pay_status_cols[::-1]), axis=1)

    # 3.4. Indicadores de Comportamiento de Pago
    # Se extraen características sobre el comportamiento de pago del cliente.
    df_features['pay_status_avg'] = df_features[pay_status_cols].mean(axis=1)
    df_features['months_with_delay'] = (df_features[pay_status_cols] > 0).sum(axis=1)
    df_features['zero_payment_months'] = (df_features[pay_amt_cols] == 0).sum(axis=1)

    print("\n--- Ingeniería de Características para Clasificación completada ---")
    display(df_features.head())


--- Ingeniería de Características para Clasificación completada ---


,ID,limit_bal,sex,education,marriage,age,pay_sept,pay_aug,pay_july,pay_june,...,utilization_april,payment_ratio_april,bill_amt_avg,bill_amt_std,pay_amt_avg,bill_amt_slope,pay_status_slope,pay_status_avg,months_with_delay,zero_payment_months
0,1,20000,2,2,1,24,2,2,-1,-1,...,0.000000,0.000000,1284.000000,1761.633219,114.833333,844.571429,0.914286,-0.333333,2,5
1,2,120000,2,2,2,26,-1,2,0,0,...,0.027175,0.613309,2846.166667,637.967841,833.333333,-247.857143,-0.257143,0.500000,2,2
2,3,90000,2,2,2,34,0,0,0,0,...,0.172767,0.321564,16942.166667,6064.518593,1836.333333,1854.714286,0.000000,0.000000,0,0
3,4,50000,2,2,1,37,0,0,0,0,...,0.590940,0.033844,38555.666667,10565.793518,1398.000000,4743.257143,0.000000,0.000000,0,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,0.382620,0.035492,18223.166667,10668.590074,9841.500000,-2231.514286,-0.171429,-0.333333,0,0


In [89]:
if df is not None:
    # Se seleccionan solo las columnas numéricas para el cálculo de VIF.
    numeric_cols_for_vif = df_features.select_dtypes(include=np.number).drop(columns=['ID', 'default_payment_next_month'])
    
    # Se añade una constante para el cálculo correcto del VIF.
    X_vif = add_constant(numeric_cols_for_vif)
    
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_vif.columns
    vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
    
    print("\n--- Análisis de Factor de Inflación de la Varianza (VIF) ---")
    # Se muestran las 15 variables con mayor VIF, que son las más problemáticas.
    display(vif_data.sort_values(by='VIF', ascending=False).head(15))

/opt/anaconda3/envs/riesgo_crediticio_env/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)



--- Análisis de Factor de Inflación de la Varianza (VIF) ---


,feature,VIF
23,pay_amt_april,inf
22,pay_amt_may,inf
21,pay_amt_june,inf
20,pay_amt_july,inf
19,pay_amt_aug,inf
18,pay_amt_sept,inf
17,bill_amt_april,inf
16,bill_amt_may,inf
15,bill_amt_june,inf
14,bill_amt_july,inf


In [90]:
if df is not None:
    features_reducidas = [
        # Demográficas y Límite de Crédito
        'limit_bal', 'sex', 'education', 'marriage', 'age',
        # Comportamiento de Pago (las más predictivas)
        'pay_sept', 'pay_status_avg', 'pay_status_slope',
        # Salud Financiera (menos correlacionadas)
        'utilization_sept', 'bill_amt_std', 'pay_amt_avg',
        # Variable Objetivo
        'default_payment_next_month'
    ]
    df_reducido = df_features[features_reducidas]
    print("\n--- Dataset Reducido Creado ---")
    display(df_reducido.head())



--- Dataset Reducido Creado ---


,limit_bal,sex,education,marriage,age,pay_sept,pay_status_avg,pay_status_slope,utilization_sept,bill_amt_std,pay_amt_avg,default_payment_next_month
0,20000,2,2,1,24,2,-0.333333,0.914286,0.195650,1761.633219,114.833333,1
1,120000,2,2,2,26,-1,0.500000,-0.257143,0.022350,637.967841,833.333333,1
2,90000,2,2,2,34,0,0.000000,0.000000,0.324878,6064.518593,1836.333333,0
3,50000,2,2,1,37,0,0.000000,0.000000,0.939800,10565.793518,1398.000000,0
4,50000,1,2,1,57,-1,-0.333333,-0.171429,0.172340,10668.590074,9841.500000,0


In [91]:
if df is not None:
    df_features_reg = df.copy()
    
    # Se usan solo datos hasta junio para evitar fuga de información.
    bill_cols_reg = ['bill_amt_june', 'bill_amt_may', 'bill_amt_april']
    pay_cols_reg = ['pay_amt_may', 'pay_amt_april']
    
    # 6.1. Ratios y Agregados
    df_features_reg['utilization_june'] = df_features_reg['bill_amt_june'] / (df_features_reg['limit_bal'] + epsilon)
    df_features_reg['bill_amt_avg_3m'] = df_features_reg[bill_cols_reg].mean(axis=1)
    df_features_reg['pay_amt_avg_2m'] = df_features_reg[pay_cols_reg].mean(axis=1)

    print("\n--- Ingeniería de Características para Regresión completada ---")



--- Ingeniería de Características para Regresión completada ---


In [92]:
if df is not None:
    try:
        processed_data_path = project_root / "data" / "processed"
        
        # Guardar dataset completo para clasificación
        path_clasificacion = processed_data_path / "features_clasificacion.csv"
        df_features.to_csv(path_clasificacion, index=False)
        print(f"\nDataset para clasificación guardado en: {path_clasificacion}")

        # Guardar dataset reducido para clasificación
        path_reducido = processed_data_path / "features_reducido_clasificacion.csv"
        df_reducido.to_csv(path_reducido, index=False)
        print(f"Dataset reducido para clasificación guardado en: {path_reducido}")
        
        # Guardar dataset para regresión
        path_regresion = processed_data_path / "features_regresion.csv"
        df_features_reg.to_csv(path_regresion, index=False)
        print(f"Dataset para regresión guardado en: {path_regresion}")

    except Exception as e:
        print(f"\nOcurrió un error al guardar los archivos: {e}")



Dataset para clasificación guardado en: /Users/edusant/Desktop/personal/blue_tab/proyecto-riesgo-crediticio/data/processed/features_clasificacion.csv
Dataset reducido para clasificación guardado en: /Users/edusant/Desktop/personal/blue_tab/proyecto-riesgo-crediticio/data/processed/features_reducido_clasificacion.csv
Dataset para regresión guardado en: /Users/edusant/Desktop/personal/blue_tab/proyecto-riesgo-crediticio/data/processed/features_regresion.csv
